##### Copyright 2022 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ABFE Tutorial

## Note

1. If you use a public colab instance, remember to change your runtime type to GPU
2. If you deployed a custom GCE runtime, please connect to a GPU VM instance.

## Summary

1. Pull Gromacs 2022.3 binary compiled specifically for Colab GPU environment and install the software.
2. Install Gromacs Python API on the fly, together with other packages used in the tutorial.
3. Run simulations for benchmarks and show how to use the official GMX
Python API to modify inputs.
4. Visualize the output conformation for the longer simulation.

In [ ]:
#@title Install Gromacs and Gromacs Python API
!wget https://storage.googleapis.com/gromacs-bin/gromacs-avx2_256-cuda-11_2.tar.gz -O /tmp/gromacs.tar.gz
!wget https://storage.googleapis.com/gromacs-bin/benchMEM.tpr -O /tmp/benchMEM.tpr
!wget https://storage.googleapis.com/gromacs-bin/hif2a_eq.tpr -O /tmp/hif2a_eq.tpr

!tar zxf /tmp/gromacs.tar.gz
!sudo rm -rf /usr/local/gromacs
!sudo mv gromacs-avx /usr/local/gromacs
!rm /tmp/gromacs.tar.gz

!pip3 install --upgrade pip
!pip3 install setuptools wheel cmake pybind11 py3DMol
!gmxapi_ROOT=/usr/local/gromacs/ pip3 install --no-cache-dir gmxapi

In [ ]:
#@title Set Level for Logging
#@markdown Sets the python logging level. This is useful for debugging.
import logging

log_level = 'DEBUG' #@param ['DEBUG', 'INFO', 'ERROR']

_log_level = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'ERROR': logging.ERROR
}

logging.basicConfig(level=_log_level[log_level])

In [ ]:
#@title [Please Rerun after Restart] Create a Session
#@markdown Please rerun this cell every time when you restart the runtime.
#@markdown A new session ID will be created for a new colab runtime session.
#@markdown Run this cell again will refresh the session ID.

import os
import time

import gmxapi as gmx


# Sets a session ID for this session.
_SESSION_ID = str(int(time.time())) #@param

# Monkey patch this session ID into the gmxapi.operation.ResourceManager.
# Hack. Do not use in the user code.
# This is temporarily added to allow users to start from a new set of working
# directories for all the mdrun launches. GMX API doesn't expose the
# underlying resource manager or context yet.
def operation_id(self):
  return f"{self._base_operation_id}_{_SESSION_ID}"
gmx.operation.ResourceManager.operation_id = property(operation_id)


def get_work_dir(md_output):
  values = [v for v in md_output.values()]
  return values[0].result()


def tail_log(md_output, output_file_base, last_n_lines=6):
  work_dir = get_work_dir(md_output)

  with open(os.path.join(work_dir, output_file_base + '.log'), 'r') as log:
    for line in log.readlines()[-last_n_lines:]:
      print(line)

print("Session ID: ", _SESSION_ID)

In [ ]:
#@title Run a Short Simulation for Benchmark

# Read a TPR sample and run it.
# The TPR file configures a 20ps simulation.
input_tpr = gmx.read_tpr('/tmp/benchMEM.tpr')

# Can specify computations to GPU.
output_file_base = 'short'
md = gmx.mdrun(input_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})

# Run the simulation
md.run()

# Tail performance
tail_log(md.output, output_file_base)

In [ ]:
#@title Modified Params and Run A Longer Simulation
# Run the simulation for 40 ps.
output_file_base = 'long2'

modified_tpr = gmx.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})
md = gmx.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

In [ ]:
#@title What about a binding affinity simulation with HIF2a?
output_file_base = 'hif'

input_tpr = gmx.read_tpr('/tmp/hif2a_eq.tpr')
modified_tpr = gmx.modify_input(
    input=input_tpr, parameters={'nsteps': 20000, 'nstlist': 100})
md = gmx.mdrun(modified_tpr, runtime_args={
    '-nb': 'gpu', '-pme': 'gpu', '-pmefft': 'gpu', '-bonded': 'gpu',
    '-deffnm': output_file_base})
md.run()

# Tail performance
tail_log(md.output, output_file_base)

## Visualization

Visualize the output from the above HIF2A simulation.
We use py3DMol to visualize the output gro file.

In [ ]:
#@title Visualize the output conformation of the HIF2A simulation.
import py3Dmol

view = py3Dmol.view()
work_dir = get_work_dir(md.output)

view.addModel(
    open(os.path.join(work_dir, output_file_base + '.gro'), 'r').read(), 'gro')

view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({}, {'cartoon': {'color': 'spectrum'}})

view.show()

# ABFE Workflow
-----

## 1. Copy input files and setup folder structure

This is the structure for each calculation.

```
ligand1/
  water/                # ligand in water simulations
    stateA/             # coupled ligand (i.e. solvated)
      run1/             # only 1 run as example, but we'll have >1
        em/             # energy minimization
        eq_posre/       # equilibration with position restraints
        eq/             # equilibrium simulation
        transitions/    # non-equilibrium transitions
          frame1/       # individual non-eq runs starting
          frame2/
          ...
          frame100/    
    stateB              # decoupled ligand (i.e. in vacuum)
      run1/
        em/
        eq_posre/
        eq/
        transitions/
  protein/              # ligand in protein simulations
    stateA/             # coupled ligand (i.e. protein-ligand complex)
      run1/
        em/
        eq_posre/
        eq/
        transitions/
    stateB/             # decoupled ligand (i.e. apo protein + restr ligand)
      run1/
        em/
        eq_posre/
        eq/
        transitions/
```

### Setup protein-ligand restraints
This step identifies the set of restraints to be used for the `protein/stateB` simulations.

--> Matteo: we might be using a new approach for hanlding the restraints in `stateA`. I need to double check with Vytas how this is done in practice with pmx.

## 2. Prepare and run equilibrium simulations
This will prepare and run all simulations in folders `em/`, `eq_posre`, and `eq`.

## 3. Prepare and run non-equilibrium simulations
This extracts N frames from the equilibrium trajectories in the `eq/` folders, puts them in the `frame*/` folders and runs a non-equilibrium simulation from each frame. Each non-equilibrium simulation interpolates between `stateA` and `stateB` (i.e., between the "coupled" and "decoupled" ligand).

## 4. Estimate binding free energy
The analysis will take in .xvg files present in each `frame*/` folder and return the estimated free energy differences.